#Import

In [18]:
!pip install pysam
!apt update && apt install samtools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 336 kB in 2s (184 kB/s)
Reading package lists... Done
Building dependency tree       
Reading s

In [19]:
!git clone https://github.com/abyzovlab/CNVpytor.git
%cd CNVpytor
!python setup.py install

Cloning into 'CNVpytor'...
remote: Enumerating objects: 1963, done.
remote: Counting objects: 100% (217/217), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 1963 (delta 139), reused 168 (delta 102), pack-reused 1746
Receiving objects: 100% (1963/1963), 138.07 MiB | 30.53 MiB/s, done.
Resolving deltas: 100% (1376/1376), done.
/content/CNVpytor/CNVpytor
running install
/usr/lib/python3.9/distutils/cmd.py:62: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/lib/python3.9/distutils/cmd.py:62: EasyIn

In [20]:
from google.colab import drive
import tarfile
import sys
import pysam
import cnvpytor
import cnvfinder
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#OpenDataset

In [21]:
import gzip
import pysam
from tqdm import tqdm
import subprocess
def convert(fastq_path,bam_dir,cell_id,max_records=1000):
  # Specify the paths to your input FASTQ and output BAM files
  bam_file_path = bam_dir+"Cell"+cell_id
  header={"HD": {"VN": "1.0", "SO": "unsorted"},
              "SQ": [
                  {"SN": "ref1|seq1", "LN": 1000000},
                  {"SN": "ref1|seq2", "LN": 1000000},
                  {"SN": "ref2|seq1", "LN": 1000000},
              ],
          }
  # Open the FASTQ file in binary mode using gzip
  with gzip.open(fastq_path, "rt") as fastq_file:
      # Create a BAM file for writing
      with pysam.AlignmentFile(bam_file_path, "wb", header=pysam.AlignmentHeader().from_dict(header)) as bam_file:
          # Use tqdm to create a progress bar
          #progress_bar = tqdm(total=4*max_records, unit="lines",dynamic_ncols=True)
          records_written = 0  # Counter for number of records converted
          for i, line in enumerate(fastq_file):
              # Update the progress bar
              #progress_bar.update(1)

              # Extract the sequence, quality scores, and other information from the FASTQ record
              if i % 4 == 0:
                  # Header line
                  read_name = line.strip()
              elif i % 4 == 1:
                  # Sequence line
                  sequence = line.strip()
              elif i % 4 == 2:
                  # Plus line (ignored)
                  pass
              elif i % 4 == 3:
                  # Quality scores line
                  quality_scores = line.strip()
                  # Create a SAM/BAM record and write it to the BAM file
                  record = pysam.AlignedSegment()
                  record.query_name = read_name
                  record.query_sequence = sequence
                  record.query_qualities = pysam.qualitystring_to_array(quality_scores)
                  bam_file.write(record)
                  records_written += 1

              # Break the loop when the maximum number of records is reached
              if records_written >= max_records:
                  break

          # Close the progress bar
          #progress_bar.close()

  # Sort and index the BAM file
  pysam.sort("-o", bam_file_path + ".bam", bam_file_path)
  pysam.index(bam_file_path + ".bam")
  print("File are written")


In [23]:
for cell_id in ["01","02","03","04","05"]:
  try:
    max_records=10000
    convert("/content/drive/MyDrive/Colab_Notebooks/StatsML/Project/src/dataset/SC-"+cell_id+".R1.fastq.gz","/content/drive/MyDrive/Colab_Notebooks/StatsML/Project/src/dataset/bam/",cell_id,max_records=max_records)
  except FileNotFoundError:
    None

File are written
File are written
File are written
File are written


In [28]:
import pysam
cell_id="02"
# Specify the path to your BAM file
bam_file_path = "/content/drive/MyDrive/Colab_Notebooks/StatsML/Project/src/dataset/bam/Cell"+cell_id+".bam"
#bam_file_path="/content/drive/MyDrive/Colab_Notebooks/StatsML/Project/data/bam/cell_idx_2.bam"
# Open the BAM file for reading with check_sq=False
with pysam.AlignmentFile(bam_file_path, "rb") as bam_file:
    # Iterate over the first 5 records in the BAM file
    for i, record in enumerate(bam_file):
        # Print the record information
        print(f"Record {i+1}:")
        print(f"Query name: {record.query_name}")
        print(f"Reference name: {record.reference_name}")
        print(f"Reference start: {record.reference_start}")
        print(f"Reference end: {record.reference_end}")
        print(f"CIGAR string: {record.cigarstring}")
        print(f"Query sequence: {record.query_sequence}")
        print(f"Query qualities: {record.query_qualities}")
        print(f"Mapping quality: {record.mapping_quality}")
        print(f"Tags: {record.tags}")
        print("---")
        if i >= 1:
            # Stop after printing the first 5 records
            break


Record 1:
Query name: @4ZN8DQ1:260:C6CY6ACXX:1:1101:1278:2073 1:N:0:AAACATC
Reference name: None
Reference start: -1
Reference end: None
CIGAR string: None
Query sequence: CAGTTCACAGGTGTGATCAGCACAACTGAAGAGTTAGACTTTGAATCCTCCCCAGAGACATATAGNTTCATTGTTAGAGCCTCTGANTGGGGGAGATCGG
Query qualities: array('B', [33, 31, 34, 37, 37, 36, 35, 37, 39, 39, 39, 37, 39, 39, 40, 40, 41, 40, 41, 41, 40, 41, 41, 41, 40, 39, 40, 41, 41, 41, 41, 38, 40, 36, 38, 34, 39, 37, 37, 35, 39, 40, 41, 40, 40, 40, 38, 38, 40, 39, 36, 39, 41, 41, 41, 37, 40, 38, 40, 41, 40, 41, 40, 41, 41, 2, 12, 26, 35, 34, 39, 39, 39, 39, 39, 36, 39, 37, 36, 37, 37, 37, 34, 36, 36, 25, 2, 11, 20, 26, 30, 31, 35, 30, 27, 24, 32, 34, 33, 35])
Mapping quality: 0
Tags: []
---
Record 2:
Query name: @4ZN8DQ1:260:C6CY6ACXX:1:1101:1428:2082 1:N:0:AAACATC
Reference name: None
Reference start: -1
Reference end: None
CIGAR string: None
Query sequence: AGTGGGTTGTCACAAATGAAGTGTCCCATAAAGCTCATTCCCTAAATGAAGAATCAGAACAGAAACTCTAGAGCCTCCCCTCTCCCCTACACACC

In [8]:
app = cnvpytor.Root("cnv.pytor", create=True)
app.rd([bam_file_path])
bin_sizes=[10, 100]
app.calculate_histograms(bin_sizes=bin_sizes)
app.partition(bin_sizes)
calls=app.call(bin_sizes)
for bin_size in calls:
    for call in calls[bin_size]:
        print(("{:13}{:>5}{:10}{:10}{:10}{:15.2f}{:15.2e}{:15.2e}").format(*tuple(call)))

In [9]:
calls

{10: [], 100: []}